# What's happening here?

4_link_video_behavior.ipynb

Run after 1_, 2_, and 3_

Made in environment **PMS_Study**

## TO DO: bekijk file PMS_analysis/2_preparation

Probeer veel met loops en functions te werken om de code overzichtelijk te houden

 * Start with file cleanData.csv (geeft informatie over van welke participant welke files bestaan)
 * Load timings.csv file that Tilia and Sofie made
 * Start loop from the top and use Order and A-B
 * In loop check if file (corresponding CSV) exists in Directory (/10001 or /10002 depedent of A/B): later refered to as behaviordata
 * In loop check if file (corresponding .WEBM) exists in Directory
 
If both exist:
- Check if output of Openface exists in Z:\shares\ghepmk_data\2020_Kappen_PMS\10002\video_features (or /10001 dependent of A/B)
- IF True:
Load this as facedata
- IF False:
Print: "Video file present for file (insert filename) but no openface present)"
Break/skip

Use filename/pptnum to search for right starting time in timings.csv variable
- Find timing for beep.wav in behaviordata. 
- Set new zero point (link between the timings.csv point and this)
- Make mean scores on a few variables (we will expand this later) for the corresponding frames of each of the trials.
- To do this, you should write a function that takes the right points of a trial (start, end, trialnum etc) and save this data to a dataframe

Compute this in a way that you would be able to add it to allPMSdata.cvs

Add it all to a new dataframe

Write to csv

In [243]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math
import time
import json
import statistics

from tqdm import tqdm
from pathlib import Path

import re

In [167]:
# Create all relevant directories
data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')
date_dir = "06102021/"
data_all_dir = os.path.join(data_dir, date_dir)

# Load base data
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")
allData = pd.read_csv(data_all_dir + "allPMSdata.csv", encoding = "ISO-8859-1")
timings = pd.read_csv(data_all_dir + "timings.csv", encoding = "ISO-8859-1")
timings = timings.rename(columns={"ï»¿ImageGroup": "ImageGroup"}, index={'ONE': 'Row_1'}) # Rename first column, because was formatted weird





In [169]:
timings

,ImageGroup,Date,pptnumber,StartTime,EndTime,OtherComments,Unnamed: 6,Exclusion,TestMoment
0,B,8/27/2020,14,0.26,0.00,"looks to the side, tv playing",NaN,1,0
1,B,9/28/2020,66,0.16,5.38,looks at phone. Same ppt as 14,NaN,1,0
2,B,8/31/2020,106,0.36,0.00,NaN,NaN,0,2
3,B,9/23/2020,119,1.14,6.47,drinking tea,NaN,0,2
4,B,07/09/2021,123,0.31,5.14,typing (texting0),NaN,0,2
...,...,...,...,...,...,...,...,...,...
520,A,8/31/2021,674,1.44,7.21,"in background, there's a picture of a face (no...",NaN,0,2
521,A,07/08/2021,676,2.08,7.05,hair bit over left eye,NaN,0,1
522,A,9/24/2021,678,0.00,0.00,no beeps,NaN,0,2
523,A,9/30/2021,679,0.12,5.11,"eyebrow piercing, chin not visible",NaN,0,2


In [171]:
# Get all OpenFace files
os.chdir(data_dir)
AFiles = glob.glob("10001/video_features/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
BFiles = glob.glob("10002/video_features/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory

# Opening JSON file with all video duration information
f = open(data_all_dir + 'video_durations.json',)
durationsdata = json.load(f)
f.close()

In [125]:
openface

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,0.063975,-0.002084,-0.997949,0.072608,-0.148678,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,0,0.001,0.98,1,0.035288,-0.006892,-0.999353,0.055273,-0.158703,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,0,0.002,0.98,1,0.024994,0.009102,-0.999646,0.052177,-0.153878,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,0,0.003,0.98,1,0.033676,-0.017344,-0.999282,0.039422,-0.162667,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,5,0,0.004,0.98,1,-0.011143,0.000171,-0.999938,-0.005280,-0.147903,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10423,10424,0,10.423,0.98,1,0.042126,-0.121335,-0.991717,-0.178407,-0.103466,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
10424,10425,0,10.424,0.98,1,0.042254,-0.120352,-0.991832,-0.178183,-0.102205,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
10425,10426,0,10.425,0.98,1,0.042256,-0.120901,-0.991765,-0.177429,-0.103008,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
10426,10427,0,10.426,0.88,1,0.042502,-0.117136,-0.992206,-0.176501,-0.099713,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [172]:
# The BIG loop where all the magic happens

# Create the columns we're gonna be using
allData["AU12"] = np.nan
allData["AU15"] = np.nan
allData["happy"] = np.nan
allData["sad"] = np.nan
allData["digust"] = np.nan
allData["anger"] = np.nan

index = 0
for pptnum in tqdm(timings["pptnumber"]): # Loop over all ppt's that had a video available (therefore present in timings.csv)
    
    print("processing pptnum: " + str(pptnum))
    idx = np.where(allData['ID'] == int(pptnum))[0]
    if timings["Exclusion"][index] == 1: # Only continue the loop if there is no Exclusion marked
        print("Exclusion file for ppt: " + str(pptnum) + " at moment: " + timings["ImageGroup"][index])
    else:
        videoOffset = float(timings["StartTime"][index])
        if videoOffset == 0: # if a timing start point was not present, it was set to 0. We don't process those because we cant sync them
            print('start time for this ppt is set to zero')
        else:
            if len(idx) < 1: # Check if occurences in datafile exist for this participant/video file
                print("No input data found for video file of pptnum: " + str(pptnum) + " with video file " + timings["ImageGroup"][index] )
            else:           
                moment = timings["ImageGroup"][index]
                if moment ==  'A':
                    indices = [i for i, s in enumerate(AFiles) if str(pptnum) in s]
                    fileDir = AFiles[indices[0]] 
                elif moment == 'B':
                    indices = [i for i, s in enumerate(BFiles) if str(pptnum) in s]
                    fileDir = BFiles[indices[0]] 
                else:
                    print("Some error with testing moment")
                    break

                print(fileDir)
                if len(indices) > 1:
                    print("Too many OpenFace files for ppt: " + str(pptnum) + " at moment " + moment)
                else: # If we get to here - the party really starts
                    openface = pd.read_csv(fileDir, encoding = "ISO-8859-1") # Read openface output csv

                    # Get corresponding video file duration 
                    splits = fileDir.split('/video_features\\')
                    fileduration = [value for key, value in durationsdata.items() if splits[0] in key.lower() and splits[1][23:len(splits[1])-4] in key.lower()] # Very weird formatting, but when checking for directory (10001/10002) and only the date (something with curly bracktes messes it up. 
                    frameduration = fileduration[0] / len(openface)
                    openface.insert (3, "timestamp2", np.arange(0, fileduration[0], frameduration).tolist()) # Add new timestamps right after the first timestamp
                    videoOffset
                    behaviorOffset = allData['startBeep'][idx[0]]
                    correctedStarts = allData['startView'][idx] - behaviorOffset # startViews - offSet
                    videoStarts = ((correctedStarts + videoOffset) / 1000).tolist() # These will be the timestamps in seconds because / 1000

                    correctedEnds = allData['stopView'][idx] - behaviorOffset # startViews - offSet
                    videoEnds = ((correctedEnds + videoOffset) / 1000).tolist()

                    # Loop over all the trials and get the appropriate values - add to allData
                    for trial in range(0,len(videoStarts)):
                        vidStartIdx = openface.iloc[(openface['timestamp2']-videoStarts[trial]).abs().argsort()[:1]].index.tolist()[0] # Get start idx in openface
                        vidEndIdx = openface.iloc[(openface['timestamp2']-videoEnds[trial]).abs().argsort()[:1]].index.tolist()[0] # Get end idx in openface

                        # Create mean value between these idx's and add to big dataframe
                        allData.loc[idx[trial], 'AU12'] = openface[' AU12_r'][vidStartIdx:vidEndIdx].mean() 
                        allData.loc[idx[trial], 'AU15'] = openface[' AU15_r'][vidStartIdx:vidEndIdx].mean() 

                        allData.loc[idx[trial], 'happy'] = statistics.mean(openface[' AU12_r'][vidStartIdx:vidEndIdx].mean(),openface[' AU06_r'][vidStartIdx:vidEndIdx].mean())
                        allData.loc[idx[trial], 'sad'] = statistics.mean(openface[' AU01_r'][vidStartIdx:vidEndIdx].mean(),openface[' AU04_r'][vidStartIdx:vidEndIdx].mean(),openface[' AU15_r'][vidStartIdx:vidEndIdx].mean())
                        allData.loc[idx[trial], 'disgust'] = statistics.mean(openface[' AU09_r'][vidStartIdx:vidEndIdx].mean(),openface[' AU15_r'][vidStartIdx:vidEndIdx].mean(),openface[' AU16_r'][vidStartIdx:vidEndIdx].mean())
                        allData.loc[idx[trial], 'anger'] = statistics.mean(openface[' AU04_r'][vidStartIdx:vidEndIdx].mean(),openface[' AU05_r'][vidStartIdx:vidEndIdx].mean(),openface[' AU07_r'][vidStartIdx:vidEndIdx].mean(),openface[' AU23_r'][vidStartIdx:vidEndIdx].mean())

                    

    index += 1 # Create counter
    if index > 10:
        break

  0%|                                                                                          | 0/525 [00:00<?, ?it/s]

processing pptnum: 14
Exclusion file for ppt: 14 at moment: B
processing pptnum: 66
Exclusion file for ppt: 66 at moment: B
processing pptnum: 106
No input data found for video file of pptnum: 106 with video file B
processing pptnum: 119
No input data found for video file of pptnum: 119 with video file B
processing pptnum: 123
No input data found for video file of pptnum: 123 with video file B
processing pptnum: 124
No input data found for video file of pptnum: 124 with video file B
processing pptnum: 125
No input data found for video file of pptnum: 125 with video file B
processing pptnum: 126
No input data found for video file of pptnum: 126 with video file B
processing pptnum: 127
start time for this ppt is set to zero
processing pptnum: 129
10002/video_features\video_{ParticipantNo}_129_9-10-2020.csv


  2%|█▍                                                                                | 9/525 [00:43<41:56,  4.88s/it]


IndexError: list index out of range

In [195]:
behaviorOffset = allData['startBeep'][idx[0]]
behaviorOffset

correctedStarts = allData['startView'][idx] - behaviorOffset # startViews - offSet
videoStarts = ((correctedStarts + videoOffset) / 1000).tolist() # These will be the timestamps in seconds because / 1000

correctedEnds = allData['stopView'][idx] - behaviorOffset # startViews - offSet
videoEnds = ((correctedEnds + videoOffset) / 1000).tolist()

In [235]:

# allData['testcolumn'][idx[trial]] = 15

In [239]:
allData["testcolumn"][0] = 15

C:\Users\mitch\AppData\Local\Temp/ipykernel_29752/673889553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allData["testcolumn"][0] = 15


In [241]:
allData.loc[0, 'testcolumn'] = 18

In [244]:
openface

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.88,1,0.084617,0.029702,-0.995971,-0.030440,0.137883,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0.001,0.98,1,0.082421,0.031089,-0.996113,-0.034821,0.137754,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0.002,0.88,1,0.082904,0.026567,-0.996203,-0.030058,0.139810,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0.003,0.88,1,0.086516,0.028193,-0.995851,-0.023438,0.142718,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0.004,0.88,1,0.086805,0.030558,-0.995757,-0.013710,0.142125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16966,16967,0,16.966,0.88,1,0.063298,-0.022907,-0.997732,-0.065511,0.067993,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16967,16968,0,16.967,0.88,1,0.061389,-0.028028,-0.997720,-0.073981,0.075372,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16968,16969,0,16.968,0.88,1,0.061555,-0.028993,-0.997683,-0.074331,0.075649,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16969,16970,0,16.969,0.88,1,0.057796,-0.026370,-0.997980,-0.074494,0.062615,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [211]:
openface.iloc[(openface[' timestamp']-videoStarts[trial]).abs().argsort()[:1]].index.tolist()

[7916]

In [204]:
for col in openface.columns:
    print(col)

frame
 face_id
 timestamp
 confidence
 success
 gaze_0_x
 gaze_0_y
 gaze_0_z
 gaze_1_x
 gaze_1_y
 gaze_1_z
 gaze_angle_x
 gaze_angle_y
 eye_lmk_x_0
 eye_lmk_x_1
 eye_lmk_x_2
 eye_lmk_x_3
 eye_lmk_x_4
 eye_lmk_x_5
 eye_lmk_x_6
 eye_lmk_x_7
 eye_lmk_x_8
 eye_lmk_x_9
 eye_lmk_x_10
 eye_lmk_x_11
 eye_lmk_x_12
 eye_lmk_x_13
 eye_lmk_x_14
 eye_lmk_x_15
 eye_lmk_x_16
 eye_lmk_x_17
 eye_lmk_x_18
 eye_lmk_x_19
 eye_lmk_x_20
 eye_lmk_x_21
 eye_lmk_x_22
 eye_lmk_x_23
 eye_lmk_x_24
 eye_lmk_x_25
 eye_lmk_x_26
 eye_lmk_x_27
 eye_lmk_x_28
 eye_lmk_x_29
 eye_lmk_x_30
 eye_lmk_x_31
 eye_lmk_x_32
 eye_lmk_x_33
 eye_lmk_x_34
 eye_lmk_x_35
 eye_lmk_x_36
 eye_lmk_x_37
 eye_lmk_x_38
 eye_lmk_x_39
 eye_lmk_x_40
 eye_lmk_x_41
 eye_lmk_x_42
 eye_lmk_x_43
 eye_lmk_x_44
 eye_lmk_x_45
 eye_lmk_x_46
 eye_lmk_x_47
 eye_lmk_x_48
 eye_lmk_x_49
 eye_lmk_x_50
 eye_lmk_x_51
 eye_lmk_x_52
 eye_lmk_x_53
 eye_lmk_x_54
 eye_lmk_x_55
 eye_lmk_y_0
 eye_lmk_y_1
 eye_lmk_y_2
 eye_lmk_y_3
 eye_lmk_y_4
 eye_lmk_y_5
 eye_lmk_y_6


In [9]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos